In [1]:
import configparser
from phidown.search import CopernicusDataSearcher
import pandas as pd
import random
# ------ config reading ------
def read_config(config_path: str) -> configparser.ConfigParser:
    """
    Reads a configuration file and returns a ConfigParser object.

    Args:
        config_path (str): Path to the configuration file.

    Returns:
        configparser.ConfigParser: Parsed configuration object.

    Raises:
        AssertionError: If the config file cannot be read.
    """
    config = configparser.ConfigParser()
    read_files = config.read(config_path)
    assert read_files, f'Failed to read config file at {config_path}'
    return config

config = read_config('/Data_large/marine/PythonProjects/SAR/sarpyx/scripts/down/config.ini')
zones = list(config["STRIPWKTS"].keys())
print(f'Available zones: {zones}')

Available zones: ['dakhla', 'guyana', 'sanpaolo', 'panama', 'houston', 'maine', 'portland', 'california', 'honololu', 'almeria', 'melilla', 'algeria', 'aden', 'bangladesh', 'taiwan', 'biscay', 'brest', 'norwich', 'englishchannel', 'northsea', 'iceland', 'amsterdam', 'durban', 'seychelles', 'portblair', 'vanuatu', 'thunderbay', 'genoa', 'miami', 'seattle', 'burlington', 'southpole', 'havana', 'britishcolumbia', 'clayton', 'westaustralia', 'norway', 'boston', 'northengland', 'portalfred', 'paraiso', 'cignectobay', 'lakeconstance', 'rotterdam']


In [2]:
def sample_products_by_month(df: pd.DataFrame, date_column: str = 'PublicationDate', max_products: int = 12) -> pd.DataFrame:
    """
    Samples up to max_products from the dataframe, each from a different month, with random years.

    Args:
        df (pd.DataFrame): DataFrame containing product metadata.
        date_column (str): Name of the column with date information.
        max_products (int): Maximum number of products to sample.

    Returns:
        pd.DataFrame: Sampled DataFrame with up to max_products, each from a different month.

    Raises:
        AssertionError: If the date_column does not exist in the DataFrame.
    """
    assert date_column in df.columns, f'Column {date_column} not found in DataFrame'
    df = df.copy()
    df['month'] = pd.to_datetime(df[date_column]).dt.month
    df['year'] = pd.to_datetime(df[date_column]).dt.year

    months = df['month'].unique().tolist()
    random.shuffle(months)
    selected_months = months[:max_products]

    sampled_rows = []
    for month in selected_months:
        month_df = df[df['month'] == month]
        years = month_df['year'].unique().tolist()
        random_year = random.choice(years)
        product_df = month_df[month_df['year'] == random_year]
        sampled_row = product_df.sample(n=1)
        sampled_rows.append(sampled_row)

    result_df = pd.concat(sampled_rows, ignore_index=True)
    assert len(result_df) <= max_products, 'Sampled more products than allowed'
    return result_df

In [3]:
Maya4strip = pd.DataFrame()

# --------- Start of the search ---------
for zone in zones:
    print(f'Searching for products in zone: {zone}')
    df = pd.DataFrame()
    aoi_wkt = config['STRIPWKTS'][zone]
    searcher = CopernicusDataSearcher()
    for swath in [1,2,3,4,5,6]:
        print(f'Searching for products in zone: {zone}, swath: {swath}')
        # Configure the search parameters
        searcher.query_by_filter(
            collection_name='SENTINEL-1',
            product_type=None,
            orbit_direction=None,
            cloud_cover_threshold=None,
            aoi_wkt=None,  # Example: aoi_wkt=aoi_wkt if you want to use the defined AOI
            start_date = '2015-05-03T00:00:00',
            end_date = '2025-05-03T04:00:00',
            top=1000,
            attributes={'processingLevel':'LEVEL0',
                        'operationalMode': 'SM',
                        'swathIdentifier': f'{swath}', # Swath identifier is: 1,2,3,4,5,6 for RAW
                        } # Swath identifier Stripmap: S1, ..., S6 SLC| 'WV1,WV2' for wavemode
        )

        df_tmp = searcher.execute_query()
        df_tmp = df_tmp.dropna(subset=['S3Path'])
        df_tmp = sample_products_by_month(df_tmp, date_column='PublicationDate', max_products=12)
        print(f'Found {len(df_tmp)} products for zone: {zone}, swath: {swath}')
        # Concatenate the results
        df = pd.concat([df, df_tmp], ignore_index=True)
    Maya4strip = pd.concat([Maya4strip, df], ignore_index=True)

Searching for products in zone: dakhla
Searching for products in zone: dakhla, swath: 1
Found 12 products for zone: dakhla, swath: 1
Searching for products in zone: dakhla, swath: 2
Found 12 products for zone: dakhla, swath: 2
Searching for products in zone: dakhla, swath: 3
Found 12 products for zone: dakhla, swath: 3
Searching for products in zone: dakhla, swath: 4
Found 12 products for zone: dakhla, swath: 4
Searching for products in zone: dakhla, swath: 5
Found 12 products for zone: dakhla, swath: 5
Searching for products in zone: dakhla, swath: 6
Found 12 products for zone: dakhla, swath: 6
Searching for products in zone: guyana
Searching for products in zone: guyana, swath: 1
Found 12 products for zone: guyana, swath: 1
Searching for products in zone: guyana, swath: 2
Found 12 products for zone: guyana, swath: 2
Searching for products in zone: guyana, swath: 3
Found 12 products for zone: guyana, swath: 3
Searching for products in zone: guyana, swath: 4
Found 12 products for zone:

In [4]:
Maya4strip

,@odata.mediaContentType,Id,Name,ContentType,ContentLength,OriginDate,PublicationDate,ModificationDate,Online,EvictionDate,S3Path,Checksum,ContentDate,Footprint,GeoFootprint,Attributes,month,year
0,application/octet-stream,bfec30dc-d416-4125-bd32-93b3a2600737,S1A_S1_RAW__0SDV_20240629T123133_20240629T1231...,application/octet-stream,1629527553,2024-06-29T13:16:01.651000Z,2024-06-29T13:22:24.981808Z,2024-11-09T12:55:34.257720Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-1/SAR/S1_RAW__0S/2024/06/29/S...,"[{'Value': '6688eb476e4ff71af9b2c1ec284866ab',...","{'Start': '2024-06-29T12:31:33.582634Z', 'End'...",geography'SRID=4326;POLYGON ((-95.3821 28.8155...,"{'type': 'Polygon', 'coordinates': [[[-95.3821...","[{'@odata.type': '#OData.CSC.StringAttribute',...",6,2024
1,application/octet-stream,32707aef-2815-4f4d-a5f0-79304d7e9f20,S1A_S1_RAW__0SDV_20250425T123127_20250425T1231...,application/octet-stream,1650762238,2025-04-25T13:16:55.340000Z,2025-04-25T13:19:46.200311Z,2025-04-25T13:19:46.200311Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-1/SAR/S1_RAW__0S/2025/04/25/S...,"[{'Value': '5df8c736297a7ff9714ce0bec4719bd5',...","{'Start': '2025-04-25T12:31:27.572594Z', 'End'...","geography'SRID=4326;POLYGON ((-95.382 28.815, ...","{'type': 'Polygon', 'coordinates': [[[-95.382,...","[{'@odata.type': '#OData.CSC.StringAttribute',...",4,2025
2,application/octet-stream,6ad8b2ce-1709-4ab1-ad1a-9cd640d716ad,S1A_S1_RAW__0SDV_20230530T123131_20230530T1231...,application/octet-stream,1627262797,2023-05-30T13:25:03.581000Z,2023-05-30T13:31:20.580057Z,2024-03-06T13:56:07.589683Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-1/SAR/S1_RAW__0S/2023/05/30/S...,"[{'Value': '871a7dce46cc0b81e941ee7020cc315d',...","{'Start': '2023-05-30T12:31:31.875237Z', 'End'...",geography'SRID=4326;POLYGON ((-95.3827 28.8149...,"{'type': 'Polygon', 'coordinates': [[[-95.3827...","[{'@odata.type': '#OData.CSC.StringAttribute',...",5,2023
3,application/octet-stream,ea0179ce-2b57-465f-bc44-0714aa7d4332,S1A_S1_RAW__0SDH_20240830T121144_20240830T1212...,application/octet-stream,1952221272,2024-08-30T13:12:09.277000Z,2024-08-30T13:18:43.986319Z,2024-08-30T13:19:00.299317Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-1/SAR/S1_RAW__0S/2024/08/30/S...,"[{'Value': 'e5e1016dd6f633660115bf354b5cf401',...","{'Start': '2024-08-30T12:11:44.935310Z', 'End'...",geography'SRID=4326;POLYGON ((-88.2841 40.6063...,"{'type': 'Polygon', 'coordinates': [[[-88.2841...","[{'@odata.type': '#OData.CSC.StringAttribute',...",8,2024
4,application/octet-stream,6ba646cc-5092-4bd0-ab57-eabba24770ce,S1A_S1_RAW__0SDV_20240921T123133_20240921T1231...,application/octet-stream,1621694096,2024-09-21T13:28:32.563000Z,2024-09-21T13:34:38.472856Z,2024-09-21T13:34:52.231952Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-1/SAR/S1_RAW__0S/2024/09/21/S...,"[{'Value': '49ac3f7b73efbd2d6f2a8c549cacf3f6',...","{'Start': '2024-09-21T12:31:33.749703Z', 'End'...",geography'SRID=4326;POLYGON ((-95.3804 28.8145...,"{'type': 'Polygon', 'coordinates': [[[-95.3804...","[{'@odata.type': '#OData.CSC.StringAttribute',...",9,2024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3163,application/octet-stream,dbfb6397-a132-4e49-ab6e-a07b4d7dcba9,S1A_S6_RAW__0SDV_20250217T214402_20250217T2144...,application/octet-stream,1447998480,2025-02-17T22:33:22.643000Z,2025-02-17T22:36:17.616326Z,2025-02-17T22:36:17.616326Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-1/SAR/S6_RAW__0S/2025/02/17/S...,"[{'Value': '6232ca1596a08fe9b6c6a3e7f698f45f',...","{'Start': '2025-02-17T21:44:02.124809Z', 'End'...",geography'SRID=4326;POLYGON ((-46.4219 -24.822...,"{'type': 'Polygon', 'coordinates': [[[-46.4219...","[{'@odata.type': '#OData.CSC.StringAttribute',...",2,2025
3164,application/octet-stream,9c85a30a-4e43-4d17-890b-d14c27c756e6,S1A_S6_RAW__0SDV_20250114T082855_20250114T0829...,application/octet-stream,1473567901,2025-01-14T09:05:39.504000Z,2025-01-14T09:07:57.625644Z,2025-01-16T13:37:42.07

In [5]:
Maya4strip.to_csv('./Maya4strip.csv', index=False)

# BASIC SEARCH

In [13]:
from phidown.search import CopernicusDataSearcher
import pandas as pd


# You can modify these parameters as needed
searcher = CopernicusDataSearcher()

for swath in [1,2,3,4,5,6]:
    # Configure the search parameters
    searcher.query_by_filter(
        collection_name='SENTINEL-1',
        product_type=None,
        orbit_direction=None,
        cloud_cover_threshold=None,
        aoi_wkt=None,  # Example: aoi_wkt=aoi_wkt if you want to use the defined AOI
        start_date = '2018-05-03T00:00:00',
        end_date = '2025-05-03T04:00:00',
        top=1000,
        attributes={'processingLevel':'LEVEL0',
                    'operationalMode': 'SM',
                    'swathIdentifier': f'{swath}', # Swath identifier is: 1,2,3,4,5,6 for RAW
                    } # Swath identifier Stripmap: S1, ..., S6 SLC| 'WV1,WV2' for wavemode
    )

    df = searcher.execute_query()
df

,@odata.mediaContentType,Id,Name,ContentType,ContentLength,OriginDate,PublicationDate,ModificationDate,Online,EvictionDate,S3Path,Checksum,ContentDate,Footprint,GeoFootprint,Attributes
0,application/octet-stream,c9ac520c-81da-4025-baf7-ea36dcffae3b,S1A_S6_RAW__0SDV_20250502T145324_20250502T1453...,application/octet-stream,1163825187,2025-05-02T15:34:14.793000Z,2025-05-02T15:37:03.913107Z,2025-05-02T15:37:03.913107Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-1/SAR/S6_RAW__0S/2025/05/02/S...,"[{'Value': 'd5cd5e0b01654a7d7a62e662d6a664cc',...","{'Start': '2025-05-02T14:53:24.472984Z', 'End'...",geography'SRID=4326;POLYGON ((55.6298 -22.0188...,"{'type': 'Polygon', 'coordinates': [[[55.6298,...","[{'@odata.type': '#OData.CSC.StringAttribute',..."
1,application/octet-stream,32b2959c-eb99-4b12-898a-6e796e0c70c1,S1A_S6_RAW__0SDV_20250502T082944_20250502T0830...,application/octet-stream,1502399321,2025-05-02T09:05:08.814000Z,2025-05-02T09:08:41.963284Z,2025-05-02T09:08:41.963284Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-1/SAR/S6_RAW__0S/2025/05/02/S...,"[{'Value': 'a8b0b0f25942e16f38d88db9842bba5e',...","{'Start': '2025-05-02T08:29:44.556435Z', 'End'...",geography'SRID=4326;POLYGON ((145.2517 18.8476...,"{'type': 'Polygon', 'coordinates': [[[145.2517...","[{'@odata.type': '#OData.CSC.StringAttribute',..."
2,application/octet-stream,19c77fc4-bac9-4660-8f65-40429e16b870,S1A_S6_RAW__0SDV_20250502T082919_20250502T0829...,application/octet-stream,1612509054,2025-05-02T09:05:16.139000Z,2025-05-02T09:08:27.628287Z,2025-05-02T09:08:27.628287Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-1/SAR/S6_RAW__0S/2025/05/02/S...,"[{'Value': 'b7b04f323520962c6ff1ee1b69da87e1',...","{'Start': '2025-05-02T08:29:19.556404Z', 'End'...",geography'SRID=4326;POLYGON ((145.5507 17.3357...,"{'type': 'Polygon', 'coordinates': [[[145.5507...","[{'@odata.type': '#OData.CSC.StringAttribute',..."
3,application/octet-stream,f1a7097d-5fab-4770-adf0-5ba19a70b2b7,S1A_S6_RAW__0SDV_20250502T082854_20250502T0829...,application/octet-stream,1533540097,2025-05-02T09:05:10.312000Z,2025-05-02T09:08:32.834366Z,2025-05-02T09:08:32.834366Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-1/SAR/S6_RAW__0S/2025/05/02/S...,"[{'Value': 'f171aef50c6fca6f5b2f4cc4cf79ed68',...","{'Start': '2025-05-02T08:28:54.556374Z', 'End'...",geography'SRID=4326;POLYGON ((145.8501 15.8238...,"{'type': 'Polygon', 'coordinates': [[[145.8501...","[{'@odata.type': '#OData.CSC.StringAttribute',..."
4,application/octet-stream,969be156-adb3-487f-bb48-434f81eee83a,S1A_S6_RAW__0SDV_20250430T214429_20250430T2144...,application/octet-stream,676495220,2025-04-30T22:26:22.843000Z,2025-04-30T22:28:53.769066Z,2025-04-30T22:28:53.769066Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-1/SAR/S6_RAW__0S/2025/04/30/S...,"[{'Value': '20830bfb6c64088b9033c07dd766bb7d',...","{'Start': '2025-04-30T21:44:29.564400Z', 'End'...",geography'SRID=4326;POLYGON ((-46.8796 -23.243...,"{'type': 'Polygon', 'coordinates': [[[-46.8796...","[{'@odata.type': '#OData.CSC.StringAttribute',..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
670,application/octet-stream,207d0515-903d-4283-a6c6-0d8d4b6f4273,S1B_S6_RAW__0SDV_20180507T064831_20180507T0648...,application/octet-stream,1171313600,2020-10-13T05:40:16.408000Z,2023-10-24T10:39:57.993510Z,2023-10-24T10:39:57.993510Z,True,9999-12-31T23:59:59.999999Z,NaN,[{}],"{'Start': '2018-05-07T06:48:31.372017Z', 'End'...",geography'SRID=4326;POLYGON ((-176.7022 -44.91...,"{'type': 'Polygon', 'coordinates': [[[-176.702...","[{'@odata.type': '#OData.CSC.StringAttribute',..."
671,application/octet-stream,39be3384-b72e-4f4d-97c5-a4ac59fd6436,S1B_S6_RAW__0SDV_20180506T115412_20180506T1154...,application/octet-stream,1139858297,2020-10-13T07:00:46.153000Z,2023-10-24T11:11:13.257424Z,2023-10-24T11:11:13.257424Z,True,9999-12-31T23:59:59.999999Z,NaN,[{}],"{'Start': '2018-05-06T11:54:12.327141Z', 'End'...",geography'SRID=4326;POLYGON ((-88.0184 40.7528...,"{'type':

# IW

In [1]:
from phidown.search import CopernicusDataSearcher
import pandas as pd


# You can modify these parameters as needed
searcher = CopernicusDataSearcher()


# Configure the search parameters
searcher._query_by_filter(
    collection_name='SENTINEL-1',
    product_type=None,
    orbit_direction=None,
    cloud_cover_threshold=None,
    aoi_wkt=None,  # Example: aoi_wkt=aoi_wkt if you want to use the defined AOI
    start_date = '2018-05-03T00:00:00',
    end_date = '2024-05-03T04:00:00',
    top=1000,
    attributes={'processingLevel':'LEVEL0',
                'operationalMode': 'IW',
                } # Swath identifier Stripmap: S1, ..., S6 SLC| 'WV1,WV2' for wavemode
    # base_url defaults to 'https://catalogue.dataspace.copernicus.eu/odata/v1/Products'
    # order_by defaults to 'ContentDate/Start desc'
)

df = searcher.execute_query()
df
# filename = df.iloc[0]['Name']  # Get the filename of the first product
# searcher.download_product(filename)

,@odata.mediaContentType,Id,Name,ContentType,ContentLength,OriginDate,PublicationDate,ModificationDate,Online,EvictionDate,S3Path,Checksum,ContentDate,Footprint,GeoFootprint,Attributes
0,application/octet-stream,416b3a11-dc36-4da0-8f15-8f3da9293821,S1A_IW_RAW__0SDV_20240503T031926_20240503T0319...,application/octet-stream,776206485,2024-05-03T05:26:43.724000Z,2024-05-03T05:32:40.240351Z,2024-05-03T05:32:49.273311Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-1/SAR/IW_RAW__0S/2024/05/03/S...,"[{'Value': 'b8e9d30b55a53ff740f9aab8f86744ba',...","{'Start': '2024-05-03T03:19:26.175072Z', 'End'...",geography'SRID=4326;POLYGON ((32.5681 -26.9927...,"{'type': 'Polygon', 'coordinates': [[[32.5681,...","[{'@odata.type': '#OData.CSC.StringAttribute',..."
1,application/octet-stream,555e7dae-c194-4d53-b7ed-4c649094c256,S1A_IW_RAW__0SDV_20240503T031901_20240503T0319...,application/octet-stream,1576582217,2024-05-03T05:27:18.754000Z,2024-05-03T05:34:07.683441Z,2024-05-03T05:34:22.844585Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-1/SAR/IW_RAW__0S/2024/05/03/S...,"[{'Value': 'dc9d54ca03df9fb3e50987185d97b0b5',...","{'Start': '2024-05-03T03:19:01.174888Z', 'End'...",geography'SRID=4326;POLYGON ((32.7208 -26.4569...,"{'type': 'Polygon', 'coordinates': [[[32.7208,...","[{'@odata.type': '#OData.CSC.StringAttribute',..."
2,application/octet-stream,98544461-2ea1-4879-8552-27ac1c95c89b,S1A_IW_RAW__0SDV_20240503T031836_20240503T0319...,application/octet-stream,1560503420,2024-05-03T05:29:32.618000Z,2024-05-03T05:36:12.730752Z,2024-05-03T05:36:26.479706Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-1/SAR/IW_RAW__0S/2024/05/03/S...,"[{'Value': '09cf1c176c929304c28d5c8598a515e5',...","{'Start': '2024-05-03T03:18:36.174522Z', 'End'...",geography'SRID=4326;POLYGON ((33.1391 -24.9622...,"{'type': 'Polygon', 'coordinates': [[[33.1391,...","[{'@odata.type': '#OData.CSC.StringAttribute',..."
3,application/octet-stream,fc887546-f6ed-4b5d-ba26-528be0924edc,S1A_IW_RAW__0SDV_20240503T031811_20240503T0318...,application/octet-stream,1549104712,2024-05-03T05:26:38.847000Z,2024-05-03T05:33:04.296306Z,2024-05-03T05:33:18.987240Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-1/SAR/IW_RAW__0S/2024/05/03/S...,"[{'Value': 'a793de47fe93b1e2532b6eb87076b4b5',...","{'Start': '2024-05-03T03:18:11.174156Z', 'End'...",geography'SRID=4326;POLYGON ((33.5475 -23.4659...,"{'type': 'Polygon', 'coordinates': [[[33.5475,...","[{'@odata.type': '#OData.CSC.StringAttribute',..."
4,application/octet-stream,718bb6a4-4d92-46ec-836b-7ea1e2a7cfd9,S1A_IW_RAW__0SDV_20240503T031746_20240503T0318...,application/octet-stream,1570692513,2024-05-03T05:27:02.122000Z,2024-05-03T05:33:38.004087Z,2024-05-03T05:33:52.821748Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-1/SAR/IW_RAW__0S/2024/05/03/S...,"[{'Value': '3104db42b8aa9c0d80ccff73c9fcedb3',...","{'Start': '2024-05-03T03:17:46.173790Z', 'End'...",geography'SRID=4326;POLYGON ((33.9467 -21.9682...,"{'type': 'Polygon', 'coordinates': [[[33.9467,...","[{'@odata.type': '#OData.CSC.StringAttribute',..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,application/octet-stream,8d52824e-80ee-46ca-a9de-53dd54c5491f,S1A_IW_RAW__0SDV_20240501T113609_20240501T1136...,application/octet-stream,1451190675,2024-05-01T12:10:55.905000Z,2024-05-01T12:17:38.771284Z,2024-05-01T12:17:51.582119Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-1/SAR/IW_RAW__0S/2024/05/01/S...,"[{'Value': '55b817d16bdbca3d98df8dddcd326b76',...","{'Start': '2024-05-01T11:36:09.885506Z', 'End'...",geography'SRID=4326;POLYGON ((-84.1394 20.8775...,"{'type': 'Polygon', 'coordinates': [[[-84.1394...","[{'@odata.type': '#OData.CSC.StringAttribute',..."
996,application/octet-stream,cc3624b6-0e19-4382-8ba2-6629dccf9064,S1A_IW_RAW__0SDV_20240501T110537_20240501T1106...,application/octet-stream,1502800645,2024-05-01T14:54:55.192000Z,2024-05-01T15:01:38.401353Z,2024-05-01T15:01:51.380764Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-1/SAR/IW_R

In [3]:
df.iloc[999]['S3Path']  # Get the filename of the first product

'/eodata/Sentinel-1/SAR/IW_RAW__0S/2024/05/01/S1A_IW_RAW__0SDV_20240501T110422_20240501T110454_053677_0684F7_2A33.SAFE'

In [8]:
from phidown.s5cmd_utils import download 

download(
    s3_path=df.iloc[999]['S3Path'],  # Replace with the actual S3 path
    output_dir="/Data_large/marine/PythonProjects/SAR/sarpyx/decompressed_data"
)

INFO: Created configuration file: .s5cfg
INFO: Downloading from: s3://eodata/Sentinel-1/SAR/IW_RAW__0S/2024/05/01/S1A_IW_RAW__0SDV_20240501T110422_20240501T110454_053677_0684F7_2A33.SAFE/*
INFO: Output directory: /Data_large/marine/PythonProjects/SAR/sarpyx/decompressed_data/S1A_IW_RAW__0SDV_20240501T110422_20240501T110454_053677_0684F7_2A33.SAFE
INFO: Running command: s5cmd --endpoint-url https://eodata.dataspace.copernicus.eu cp s3://eodata/Sentinel-1/SAR/IW_RAW__0S/2024/05/01/S1A_IW_RAW__0SDV_20240501T110422_20240501T110454_053677_0684F7_2A33.SAFE/* /Data_large/marine/PythonProjects/SAR/sarpyx/decompressed_data/S1A_IW_RAW__0SDV_20240501T110422_20240501T110454_053677_0684F7_2A33.SAFE/


True

In [ ]:
from phidown.search import CopernicusDataSearcher
from phidown.s5cmd_utils import download 


searcher = CopernicusDataSearcher()
df = searcher.query_by_name(
    'S1A_S1_RAW__0SDV_20240629T123133_20240629T123159_054538_06A32D_8408.SAFE'
    )


download(
    s3_path=df.iloc[0]['S3Path']  # Get the S3 path of the product,
    output_dir="/Data_large/marine/PythonProjects/SAR/sarpyx/data/maya4ps"
)


INFO:phidown.s5cmd_utils:Downloading from: s3://eodata/Sentinel-1/SAR/S1_RAW__0S/2024/06/29/S1A_S1_RAW__0SDV_20240629T123133_20240629T123159_054538_06A32D_8408.SAFE/*
INFO:phidown.s5cmd_utils:Output directory: /Data_large/marine/PythonProjects/SAR/sarpyx/data/maya4ps/S1A_S1_RAW__0SDV_20240629T123133_20240629T123159_054538_06A32D_8408.SAFE
INFO:phidown.s5cmd_utils:Running command: s5cmd --endpoint-url https://eodata.dataspace.copernicus.eu cp s3://eodata/Sentinel-1/SAR/S1_RAW__0S/2024/06/29/S1A_S1_RAW__0SDV_20240629T123133_20240629T123159_054538_06A32D_8408.SAFE/* /Data_large/marine/PythonProjects/SAR/sarpyx/data/maya4ps/S1A_S1_RAW__0SDV_20240629T123133_20240629T123159_054538_06A32D_8408.SAFE/


True